In [ ]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import pandas as pd
import math
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2Model

In [ ]:
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
data_dir = '/content/gdrive/MyDrive'
filename = 'p2_train.csv'
test_filename = 'p2_test.csv'
!cp $data_dir/$filename .

In [ ]:
df_train = pd.read_csv(f'{data_dir}/{filename}')[['text', 'label']]
df_train

,text,label
0,longlost pictur archiv celebr glamour seneg wr...,0
1,advanc understand natur world respons current ...,1
2,itali test western economi bear almost total s...,1
3,south africa box granni juke jab way healthier...,1
4,coronaviru pandem nation tragedi hundr thousan...,1
...,...,...
1700,bodi pile morgu iran feel strain coronaviru cn...,0
1701,bbc bank england consid introduct electron ban...,0
1702,need know coronaviru thursday march version st...,0
1703,transcript return transcript main page cnn tra...,0


In [ ]:
df_test = pd.read_csv(f'{data_dir}/{test_filename}')[['text', 'label']]
df_test

,text,label
0,latest headlin cnn busi great shutdown newsroo...,0
1,china japan bud relationship time coronaviru t...,0
2,coronaviru could hurt boe max crisi new york c...,0
3,hong kong protest unrest crimin gener gener cr...,0
4,new art district aim brighten chennai commun d...,0
...,...,...
421,suggest polit affili play role afraid peopl co...,1
422,stephen sondheim american compos lyricist one ...,1
423,ahm janko survivor brutal boko haram attack no...,1
424,step invest contact trace expand test today us...,1


In [ ]:
X_train = df_train['text']
y_train = df_train['label']

X_test = df_test['text']
y_test = df_test['label']

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1705,), (426,), (1705,), (426,))

In [ ]:
PAD_TOKEN = "<|pad|>"
EOS_TOKEN = "<|endoftext|>"

MAX_LENGTH = 100
# this will download and initialize the pre trained tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
    pad_token=PAD_TOKEN,
    eos_token=EOS_TOKEN,
    max_length=MAX_LENGTH,
    is_split_into_words=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:

X_train = [str(ex) + EOS_TOKEN for ex in X_train]
X_test = [str(ex) + EOS_TOKEN for ex in X_test]

In [ ]:
X_train_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)['input_ids'] for x in X_train]
X_test_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)['input_ids'] for x in X_test]

X_train_in = tf.squeeze(tf.convert_to_tensor(X_train_), axis=1)
X_test_in = tf.squeeze(tf.convert_to_tensor(X_test_), axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:

X_train_in[0]

<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([ 6511, 33224,  2862,   333,  3934,   452,  4681, 27240,   454,
        3308,  1533,  3194,   267, 13034,   289,   692,   392,   269,
       20471, 29445,   636,    72, 33070,  1402,  1097, 13986,   489,
          72,  6576,  1450, 12705,  4676, 26617,  1790,  4190,  2415,
        1302,   410,   274,  8957,  3912,  6576, 23568, 31172,   686,
        1362,   288,   292,   346,  6862, 27240,   273,  4590,  3308,
        1533,   743,  2823,  2042,  2330,   435,   452,  3124,   649,
        2922,  2739,  8408,  3590,  3181,  1657,   890,  2626,  2469,
        1043,  1363,  7521, 21266,  4681,  1352,    72,  2862,   333,
        1204,  7421,  6580, 37189,   954,   380,  1182,  3812,  3485,
        7633,    72, 46754,  1097,  3952, 12198,   312,   686,  1362,
         288], dtype=int32)>

In [ ]:
X_train_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)["attention_mask"] for x in X_train]
X_test_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)["attention_mask"] for x in X_test]

X_train_mask = tf.squeeze(tf.convert_to_tensor(X_train_mask_), axis=1)
X_test_mask = tf.squeeze(tf.convert_to_tensor(X_test_mask_), axis=1)
     


In [ ]:

model = TFGPT2Model.from_pretrained("gpt2", use_cache=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id)
model.training = True

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [ ]:

model.resize_token_embeddings(len(tokenizer))

In [ ]:

for layer in model.layers:
    layer.trainable = False

In [ ]:

model.summary()

Model: "tfgpt2_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 124440576 
 r)                                                              
                                                                 
Total params: 163,038,720
Trainable params: 38,598,144
Non-trainable params: 124,440,576
_________________________________________________________________


In [ ]:

input = tf.keras.layers.Input(shape=(None,), dtype='int32')
mask = tf.keras.layers.Input(shape=(None,), dtype='int32')
x = model(input, attention_mask=mask)
#x = x.last_hidden_state[:, -1]
x = tf.reduce_mean(x.last_hidden_state, axis=1)
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)

In [ ]:

clf = tf.keras.Model([input, mask], output)

In [ ]:
clf.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 tfgpt2_model (TFGPT2Model)     TFBaseModelOutputWi  124440576   ['input_1[0][0]',                
                                thPastAndCrossAtten               'input_2[0][0]']                
                                tions(last_hidden_s                                               
                                tate=(None, None, 7                                           

In [ ]:
base_learning_rate = 0.0005
optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
#loss=tf.keras.losses.BinaryCrossentropy()
loss=tf.keras.losses.SparseCategoricalCrossentropy()

clf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
     


In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(
        monitor="accuracy", verbose=1, patience=3, restore_best_weights=True)
     


In [ ]:

y_train_in = tf.constant(y_train, dtype=tf.int32)
y_test_in = tf.constant(y_test, dtype=tf.int32)

In [ ]:

tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [ ]:
history = clf.fit([X_train_in, X_train_mask], y_train_in, epochs=20, batch_size=32, validation_split=0.2, callbacks=callbacks)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


43/43 [==============================] - 42s 968ms/step - loss: 4.1329 - accuracy: 0.3695 - val_loss: 0.6574 - val_accuracy: 0.6364
Epoch 2/20
43/43 [==============================] - 42s 961ms/step - loss: 2.1159 - accuracy: 0.5249 - val_loss: 0.5876 - val_accuracy: 0.7478
Epoch 3/20
43/43 [==============================] - 40s 944ms/step - loss: 0.9524 - accuracy: 0.6144 - val_loss: 0.5660 - val_accuracy: 0.7625
Epoch 4/20
43/43 [==============================] - 41s 946ms/step - loss: 0.6748 - accuracy: 0.6650 - val_loss: 0.5113 - val_accuracy: 0.7771
Epoch 5/20
43/43 [==============================] - 38s 871ms/step - loss: 0.5933 - accuracy: 0.7001 - val_loss: 0.4682 - val_accuracy: 0.7889
Epoch 6/20
43/43 [==============================] - 38s 883ms/step - loss: 0.5211 - accuracy: 0.7361 - val_loss: 0.4339 - val_accuracy: 0.7918
Epoch 7/20
43/43 [==============================] - 37s 870ms/step - loss: 0.4813 - accuracy: 0.7603 - val_loss: 0.4251 - val_accuracy: 0.8006
Epoch 8/20

In [ ]:
clf.evaluate([X_test_in, X_test_mask], y_test_in)

14/14 [==============================] - 4s 257ms/step - loss: 0.6329 - accuracy: 0.8333


[0.632915198802948, 0.8333333134651184]

In [ ]:

clf.training = False
y_pred = clf.predict([X_test_in, X_test_mask])

14/14 [==============================] - 3s 247ms/step


In [ ]:

y_pred_out = tf.math.argmax(y_pred, axis=-1)
y_pred_out

<tf.Tensor: shape=(426,), dtype=int64, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
     

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
     

In [ ]:
print(classification_report(y_test_in, y_pred_out))


              precision    recall  f1-score   support

           0       0.79      0.87      0.83       194
           1       0.88      0.80      0.84       232

    accuracy                           0.83       426
   macro avg       0.83      0.84      0.83       426
weighted avg       0.84      0.83      0.83       426

